In [1]:
from datasets import load_dataset, DatasetDict, load_metric, load_from_disk, Dataset
import pandas as pd

In [2]:
%cd Models/FiD

/home/ubuntu/QuestionAnswering/Notebooks/Models/FiD


In [3]:
df = pd.read_excel('/home/ubuntu/QuestionAnswering/WOT_extractive_final.xlsx').rename(columns={'Unnamed: 0': 'id'})
df['id'] = df['id'].astype(str)

# FiD specific
df['answers'] = df['output'].apply(lambda x: [x])
df['history'] = df['history'].fillna(' ')
df['steps'] = df['steps'].apply(lambda x: x[2:-2].split("\', \'"))

df['ctxs1'] = df.apply(lambda row: [{'title': 'conversation history', 'text': row['history']}], axis = 1)
df['ctxs1'] = df.apply(lambda row: row['ctxs1'] + [{'title': f'{row["title"]} description', 'text': row['description']}] , axis = 1)
df['ctxs1'] = df.apply(lambda row: row['ctxs1'] + [{'title': f'{row["title"]} ingredients', 'text': row['ingredients']}] , axis = 1)
df['ctxs1'] = df.apply(lambda row: row['ctxs1'] + [{'title': f'{row["title"]} Step {i+1}', 'text': row['steps'][i]} for i in range(len(row['steps']))] , axis = 1)


def split_context(context):
    context = context.split(' ')
    start_points = list(range(0,len(context), 200))
    end_points = list(range(250,len(context), 200))
    out = [context[start_points[i]:end_points[i]] for i in range(len(end_points))]
    #append proper ending sequence
    out.append(context[-250:])
    out = [' '.join(el) for el in out]
    return out

df['ctxs2'] = df.apply(lambda row: f"{row['context']}\nHistory: {row['history']}\n", axis=1)
df['ctxs2'] = df['ctxs2'].apply(split_context)
df['ctxs2'] = df.apply(lambda row: [{'title': f'{row["title"]} snippet {i}', 'text': row['ctxs2'][i]} for i in range(len(row['ctxs2']))] , axis = 1)

df['ctxs'] = df['ctxs1']
df['target'] = df['output']
df = df[['id', 'question', 'target', 'answers', 'ctxs', 'data_split', 'domain', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable']].fillna('')

df_train = df[df['data_split']=='train'].drop(columns=['data_split'])
df_test = df[df['data_split']=='test'].drop(columns=['data_split'])
df_val = df[df['data_split']=='validation'].drop(columns=['data_split'])

ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)
ds_val = Dataset.from_pandas(df_val)

ds = DatasetDict({'train': ds_train,
                  'test': ds_test,
                  'validation': ds_val})
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'target', 'answers', 'ctxs', 'domain', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable', '__index_level_0__'],
        num_rows: 248
    })
    test: Dataset({
        features: ['id', 'question', 'target', 'answers', 'ctxs', 'domain', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable', '__index_level_0__'],
        num_rows: 412
    })
    validation: Dataset({
        features: ['id', 'question', 'target', 'answers', 'ctxs', 'domain', 'is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation', 'is_unanswerable', '__index_level_0__'],
        num_rows: 167
    })
})


In [4]:
ds['train'].to_json("./WoT_extract_train.jsonl")
ds['test'].to_json("./WoT_extract_test.jsonl")
ds['validation'].to_json("./WoT_extract_eval.jsonl")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

1015343

## Untuned

In [5]:
! python test_reader.py \
        --model_path //home/ubuntu/QuestionAnswering/Notebooks/Models/FiD/pretrained_models/nq_reader_base \
        --eval_data ./WoT_extract_test.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_0Shot \
        --checkpoint_dir checkpoint

2023-03-25 12:01:37.542829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 12:01:40.011556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

## Finetune

In [5]:
! python train_reader.py \
        --train_data ./WoT_extract_train.jsonl \
        --eval_data ./WoT_extract_eval.jsonl \
        --model_size base \
        --per_gpu_batch_size 1 \
        --n_context 20 \
        --total_step 5000 \
        --warmup_step 500 \
        --name WoT_extractive_context1\
        --checkpoint_dir /home/ubuntu/QuestionAnswering/Notebooks/Models/FiD/pretrained_models/nq_reader_base


2023-03-25 12:45:53.440878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 12:45:54.088790: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

In [6]:
! python test_reader.py \
        --model_path /home/ubuntu/QuestionAnswering/Notebooks/Models/FiD/pretrained_models/nq_reader_base/WoT_extractive_context1/checkpoint/best_dev \
        --eval_data ./WoT_extract_test.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 20 \
        --name WoT_extractive_context1 \
        --checkpoint_dir checkpoint \
        --write_inference

2023-03-25 13:13:02.923923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 13:13:03.573487: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

In [7]:
model_dir = '/home/ubuntu/QuestionAnswering/Notebooks/Models/FiD/'

with open('/home/ubuntu/QuestionAnswering/Notebooks/Models/FiD/inference.txt', 'r') as f:
    lines = f.readlines()

predictions = {line.strip().split(' || ')[0]: line.strip().split(' || ')[1] for line in lines}

df_test['predictions'] = df_test['id'].apply(lambda x: predictions[x])

df_test['output'] = df_test['target']

df_test['EM'] = df_test['predictions'] == df_test['output']
df_test['pred_in_exp'] = df_test.apply(lambda row: len(row['predictions']) / len(row['output']) if row['predictions'] in row['output'] else 0, axis=1)
df_test['exp_in_pred'] = df_test.apply(lambda row: len(row['output']) / len(row['predictions']) if row['output'] in row['predictions'] else 0, axis=1)

print(f"Exact Match: {df_test['EM'].mean()}")
print(f"Prediction in Epxectation: {df_test['pred_in_exp'].mean()}")
print(f"Epxectation in Prediction: {df_test['exp_in_pred'].mean()}")

df_test.to_excel(model_dir + 'test_set_predictions.xlsx')

with open(model_dir + 'test_evaluation.txt', 'a') as f:
  f.write('Class|elements in class|EM|elements in cooking|EM cooking|elements in diy|EM diy\n')
  for q_class in ['is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation']:
    df_t_cooking = df_test[(df_test[q_class] == True) & (df_test['domain']=='cooking')] 
    df_t_diy = df_test[(df_test[q_class] == True) & (df_test['domain']=='diy')]
    df_t = df_test[df_test[q_class] == True]
    f.write(f'{q_class}|{len(df_t)}|{df_t["EM"].sum()/len(df_t)}|{len(df_t_cooking)}|{df_t_cooking["EM"].sum()/len(df_t_cooking)}|{len(df_t_diy)}|{df_t_diy["EM"].sum()/len(df_t_diy)}\n')
  f.write('\n\n')
  
  f.write('Class|elements in class|Pred_in_Exp|elements in cooking|Pred_in_Exp cooking|elements in diy|Pred_in_Exp diy\n')
  for q_class in ['is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation']:
    df_t_cooking = df_test[(df_test[q_class] == True) & (df_test['domain']=='cooking')] 
    df_t_diy = df_test[(df_test[q_class] == True) & (df_test['domain']=='diy')]
    df_t = df_test[df_test[q_class] == True]
    f.write(f'{q_class}|{len(df_t)}|{df_t["pred_in_exp"].mean()}|{len(df_t_cooking)}|{df_t_cooking["pred_in_exp"].mean()}|{len(df_t_diy)}|{df_t_diy["pred_in_exp"].mean()}\n')
  f.write('\n\n')
  
  f.write('Class|elements in class|Exp_in_Pred|elements in cooking|Exp_in_Pred cooking|elements in diy|Exp_in_Pred diy\n')
  for q_class in ['is_factoid', 'is_confirmation', 'is_complex', 'is_causal', 'is_listing', 'is_history', 'is_navigation']:
    df_t_cooking = df_test[(df_test[q_class] == True) & (df_test['domain']=='cooking')] 
    df_t_diy = df_test[(df_test[q_class] == True) & (df_test['domain']=='diy')]
    df_t = df_test[df_test[q_class] == True]
    f.write(f'{q_class}|{len(df_t)}|{df_t["exp_in_pred"].mean()}|{len(df_t_cooking)}|{df_t_cooking["exp_in_pred"].mean()}|{len(df_t_diy)}|{df_t_diy["exp_in_pred"].mean()}\n')
  f.write('\n\n')
  
  f.write(str(df_test.groupby('domain').sum().reset_index()[['domain', 'EM']]))
  

Exact Match: 0.11407766990291263
Prediction in Epxectation: 0.1371441504901909
Epxectation in Prediction: 0.1456660606635142


/tmp/ipykernel_45551/242175777.py:47: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  f.write(str(df_test.groupby('domain').sum().reset_index()[['domain', 'EM']]))


## Few Shot Experiment

In [11]:
! python train_reader.py \
        --train_data /content/WoT_extract_test.jsonl \
        --eval_data /content/WoT_extract_eval.jsonl \
        --model_size base \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --total_step 5000 \
        --warmup_step 500 \
        --name WoT_extractive_finetune \
        --checkpoint_dir /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base_few_shot

2023-02-28 23:08:12.020264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 23:08:13.343918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:08:13.344078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:08:13.344102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

### Tuned

In [12]:
! python test_reader.py \
        --model_path /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base_few_shot/WoT_extractive_finetune/checkpoint/best_dev \
        --eval_data /content/WoT_extract_train.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_finetuned \
        --checkpoint_dir checkpoint

2023-02-28 23:29:18.534971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 23:29:19.484607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:29:19.484765: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:29:19.484786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

### Untuned

In [ ]:
! python test_reader.py \
        --model_path /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base \
        --eval_data /content/WoT_extract_train.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_0Shot \
        --checkpoint_dir checkpoint

2023-02-27 16:28:03.665262: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:28:04.582474: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:28:04.582591: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:28:04.582611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War